In [218]:
import pandas as pd
import numpy as np
import re
import ast
import random

In [219]:
df = pd.read_csv("train.csv")
df =df[["title","original_language","popularity","release_date","production_companies",'production_countries','crew']]
df = df.dropna()

In [220]:
df.loc[:,["production_companies"]] = (df.loc[:,["production_companies"]]).applymap(lambda x: ast.literal_eval(x))
df.loc[:,["production_companies"]] = (df.loc[:,["production_companies"]]).applymap(lambda x: " and ".join([i["name"] for i in [j for j in x]]))

In [221]:
df.loc[:,["production_countries"]] = (df.loc[:,["production_countries"]]).applymap(lambda x: ast.literal_eval(x))
df.loc[:,["production_countries"]] = (df.loc[:,["production_countries"]]).applymap(lambda x: " and ".join([i["iso_3166_1"] for i in [j for j in x]]))

In [222]:
df.loc[:,["crew"]] = (df.loc[:,["crew"]]).applymap(lambda x: ast.literal_eval(x))
df.loc[:,["crew"]] = (df.loc[:,["crew"]]).applymap(lambda x: " and ".join([i["name"] for i in [j for j in x] if i["job"] == "Director"]))

In [223]:
df = df.rename(columns = {"crew":"director"})

In [224]:
patt = r'\d+/\d+/(\d+)'
df.loc[:,["release_date"]] = df.loc[:,["release_date"]].applymap(lambda x:"20"+re.match(patt,x).group(1) if int(re.match(patt,x).group(1)) <= 19 else "19"+re.match(patt,x).group(1))

In [225]:
df = df.rename(columns={"release_date":"release_year"})

In [226]:
df.loc[:,["popularity"]] = df.loc[:,["popularity"]].applymap(lambda x:"{:.3f}".format(float(x))) 

In [227]:
df = df.sample(210, random_state = 42)

In [228]:
def resetEnTT(row):
    inds = list(row.index)
    attri_num = random.randint(1,3)
    left = list(set(inds) - set(["title"]))
    attri_thrown = random.sample(left,k=attri_num)
    for i in range(len(inds)):
        if inds[i] in attri_thrown:
            row.loc[inds[i]] = None
    return row

In [229]:
first = df[:150]

In [230]:
same = first.sample(n=90,random_state = 42)

In [231]:
second = df[150:]

In [232]:
second = pd.concat([second,same])

In [233]:
first = first.apply(lambda x: resetEnTT(x),axis=1)
second = second.apply(lambda x: resetEnTT(x),axis=1)

In [234]:
second.columns = ["movie_name","language","box_office","year","companies","countries","director"]

In [235]:
first.to_csv("first.csv",sep=";",index_label="id",na_rep="NONE")

In [236]:
second.to_csv("second.csv",sep=";",index_label="id",na_rep="NONE")

In [237]:
first.count()

title                   150
original_language        88
popularity              104
release_year             98
production_companies     94
production_countries    115
director                106
dtype: int64

In [238]:
second.count()

movie_name    150
language      100
box_office     98
year           98
companies     101
countries     106
director       91
dtype: int64